In [1]:
from catboost import Pool, CatBoostClassifier
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
df = pd.read_csv('data.csv')

In [ ]:
df = df.drop(['customerID'], axis=1)

In [ ]:
df.head()

In [ ]:
le = preprocessing.LabelEncoder()

cols = []
for col in df.columns:
    if df[col].dtype == 'object':
        cols.append(col)
df[cols] = df[cols].apply(le.fit_transform)
df.head()

In [ ]:
df.info()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['Churn'], axis=1), 
                                                    df['Churn'], test_size=0.30, 
                                                    random_state=42, stratify=df['Churn'])

In [ ]:
X_train.head()

# Testing Default Params

In [ ]:
lgbm = lgb.LGBMClassifier()
lgbm.fit(X_train, y_train)
y_pred = lgbm.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
cbr = CatBoostClassifier(logging_level='Silent')
cbr.fit(X_train, y_train)
у_pred = cbr.predict(X_test)
print(classification_report(y_test, у_pred))

In [ ]:
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
у_pred = xgb.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
skl = GradientBoostingClassifier().fit(X_train, y_train)
у_pred = skl.predict(X_test)
print(classification_report(y_test, y_pred))

# Testing GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV

def make_grid_search(model, grid, X=X_train, y=y_train):
    clf = GridSearchCV(model, grid, verbose = 1)
    clf.fit(X_train, y_train)
    у_pred = clf.predict(X_test)
    print(clf.best_params_)
    print(classification_report(y_test, у_pred))

In [ ]:
grid = {'learning_rate': [0.03, 0.04, 0.045, 0.47, 0.403],
        'depth': [4, 6, 8, 10, 15],
        'l2_leaf_reg': [2, 3, 4, 5, 7, 10, 12],
        'iterations': [250, 300, 500]}
cbc = CatBoostClassifier(logging_level='Silent')
make_grid_search(cbc, grid, X=X_train, y=y_train)

In [ ]:
grid = {'learning_rate': [0.03, 0.1],
        'max_depth': [-1, 4, 6, 8],
        'reg_lambda': [1, 3, 5, 7, 9]}
lgbm = lgb.LGBMClassifier()
make_grid_search(lgbm, grid, X=X_train, y=y_train)

In [ ]:
grid = {'learning_rate': [0.03, 0.1],
        'n_estimators': [100, 200, 500],
        'max_depth': [1, 4, 6, 8],
        'subsample':[0.2, 0.5, 0.7]}
skl = GradientBoostingClassifier()
make_grid_search(skl, grid, X=X_train, y=y_train)

In [ ]:
grid = {'learning_rate': [0.03, 0.1],
        'n_estimators': [100, 200, 500],
        'max_depth': [1, 4, 6, 8],
        'subsample':[0.2, 0.5, 0.7]}
xgb = XGBClassifier(verbosity=0, use_label_encoder=False)
make_grid_search(xgb, grid, X=X_train, y=y_train)